<a href="https://colab.research.google.com/github/Gibur13/Digital-Architecture-for-Distance-Based-Sorting/blob/main/5_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **5. Quantization**


## 5.0 Setup Capabilities

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a2-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a2-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a2-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a2-Gibur13' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a2-Gibur13
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 9 (delta 5), reused 5 (delta 5), pack-reused 4 (from 1)
Unpacking objects: 100% (9/9), 14.61 KiB | 20.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a2-Gibur13
   baa4936..912b8e3  main       -> origin/main
Updating baa4936..912b8e3
Fast-forward
 2_size_estimator_and_profiler.ipynb |  132 ++--
 3_training_and_analysis.ipynb       |   46 +-
 5_quantization.ipynb                | 2285 ++++++++++++++++++++++++++++---------------------------
 3 files changed, 1270 insertions(+), 1193 deletions(-)
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

Please verify the cell below prints out the github repository.

In [5]:
!ls '{PROJECT_ROOT}'

1_audio_preprocessing.ipynb	     5_quantization.ipynb	      README.md
2_size_estimator_and_profiler.ipynb  6_pruning.ipynb		      src
3_training_and_analysis.ipynb	     arduino_nano_33_ble_tutorial.md  tests
4_model_conversion.ipynb	     images


### Install required packages

In [6]:
# Install libraries
!pip install tqdm
!pip install torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

### Import code dependencies

In [7]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import copy
import os

import sys

# Adding assignment 2 to the system path
# -- make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)

# Import data_proc to use data processing functions
import src.data_proc as data_proc

# Import constants to use constants defined for training
from src.constants import *

# Set random seed
# Make sure the shuffling and picking is deterministic
# Note that different value of random_seed may change rate of variation in loss/accuracy during training
# Using the same random seed value every time you rerun the notebook will
# reproduce the training and testing results
random_seed = RANDOM_SEED
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

Model folders are created, 
PyTorch models will be saved in /content/gdrive/MyDrive/ece5545/models/torch_models, 
ONNX models will be saved in /content/gdrive/MyDrive/ece5545/models/onnx_models, 
TensorFlow Saved Models will be saved in /content/gdrive/MyDrive/ece5545/models/tf_models, 
TensorFlow Lite models will be saved in /content/gdrive/MyDrive/ece5545/models/tflite_models, 
TensorFlow Lite Micro models will be saved in /content/gdrive/MyDrive/ece5545/models/micro_models.


## 5.1 Define Quantization Functions

There are some test cases in the `tests` folder to verify basic functionality of your implemented functions--these will be run automatically every time you check in your code. Additionally, we've left some simple tests in this notebook as well for you to try things out.



#### TODO 0: Implement the backward pass of `ste_round` function in `src/quant.py`

In [ ]:
# add a test if you like. There's already one under tests/


#### TODO 1: Implement the `linear_quantize` function in `src/quant.py`


In [8]:
from src.quant import linear_quantize

# Mini test case for linear_quantize
with torch.no_grad():
    x = torch.tensor([2, -0.5, 0., 1.])
    scale = 1
    zero = 0
    y = linear_quantize(x, scale, zero)
    print(y)

tensor([2., -0., 0., 1.])


#### TODO 2: Implement the `SymmetricQuantFunction` forward function in `src/quant.py`


In [9]:
from src.quant import SymmetricQuantFunction

quant_f = SymmetricQuantFunction.apply

x = torch.tensor([2, -0.5, 0., 1.])
x.requires_grad = True
bw = 2
y = quant_f(x, bw, scale, zero)
(y ** 2).sum().backward()

print(y)
print(x.grad)

tensor([1., -0., 0., 1.], grad_fn=<SymmetricQuantFunctionBackward>)
tensor([2., -0., 0., 2.])


#### TODO 3: Implement the `AsymmetricQuantFunction` forward function in `src/quant.py`

In [10]:
from src.quant import AsymmetricQuantFunction

quant_f = AsymmetricQuantFunction.apply

x = torch.tensor([2, -0.5, 0., 1.])
x.requires_grad = True
bw = 2
y = quant_f(x, bw, scale, zero)
(y ** 2).sum().backward()

print(y)
print(x.grad)

tensor([2., -0., 0., 1.], grad_fn=<AsymmetricQuantFunctionBackward>)
tensor([4., -0., 0., 2.])


#### TODO 4: Finish the Implement of `get_quantization_params` function in `src/quant.py`

In [11]:
from src.quant import QConfig

qconfig = QConfig(quant_bits=4, is_symmetric=True)
print(qconfig.get_quantization_params(x.min(), x.max()))

qconfig = QConfig(quant_bits=4, is_symmetric=False)
print(qconfig.get_quantization_params(x.min(), x.max()))

(tensor(0.2857), tensor(0))
(tensor(0.1667), tensor(3, dtype=torch.int32))


#### TODO 5: Implement the `quantize_weights_bias` function in `src/quant.py`

In [12]:
from src.quant import quantize_weights_bias, QConfig

qconfig = QConfig(quant_bits=4, is_symmetric=True)

w1 = nn.Parameter(torch.tensor([2, -0.5, 0., 1.]))
qw1 = quantize_weights_bias(w1, qconfig)
print(qw1.data)

w2 = nn.Parameter(torch.tensor([2.5, -1, 0., 1.5]))
qw2 = quantize_weights_bias(w2, qconfig)
print(qw2.data)

tensor([ 7., -2.,  0.,  4.])
tensor([ 7., -3.,  0.,  4.])


## 5.2 Quantization Function for Linear and Convolution Layer

#### TODO 6: Finish the implementation of `conv2d_linear_quantized` function in `src/quant.py`

In [13]:
import torch.nn as nn
from src.quant import QuantWrapper

layer = nn.Linear(2, 2)
layer.weight.data = torch.tensor([[0.1, 0.1], [-0.1, 0.1]]).view(2, 2).float()
layer.bias.data = torch.tensor([1, 2]).view(*layer.bias.shape).float()
x = torch.tensor([[0., 1]])
print(layer(x))

quant_layer = QuantWrapper(
    layer,
    QConfig(quant_bits=4, is_symmetric=True),
    QConfig(quant_bits=4, is_symmetric=True),
    QConfig(quant_bits=4, is_symmetric=True))
print(quant_layer(x))


tensor([[1.1000, 2.1000]], grad_fn=<AddmmBackward0>)
tensor([[1.2429, 2.1000]], grad_fn=<AddmmBackward0>)


## 5.3 Prepare model for QAT (Quantization Aware Training)

### Get Audio Processor, Devices, Data Loader, and Model

NOTE: This is identical to section 2.2 .

In [21]:
# Create audio_processor
# DATASET_DIR is defined in constants.py
# HINT: In case loading data takes too long, move the dataset from gdrive to /content/ and change the path accordingly.
audio_processor = data_proc.AudioProcessor(data_dir=DATASET_DIR)
print("Audio_processor created")

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} to run the training scrpit.')

# Define data loaders
from src.loaders import make_data_loaders
data_loaders = make_data_loaders(audio_processor, device)
train_loader = data_loaders['training']
test_loader = data_loaders['testing']
valid_loader = data_loaders['validation']

# Create a full precision (float32) TinyConv model
from src.networks import TinyConv
model_fp32 = TinyConv(model_settings=audio_processor.model_settings, \
    n_input=1, n_output=audio_processor.num_labels)
torch.save(model_fp32.state_dict(), os.path.join(TORCH_DIR, "tinyconv_float32_init_seed0_90.28%_0.pt"))
model_fp32

Audio_processor created
Using cpu to run the training scrpit.
Train size: 10556 Val size: 1333 Test size: 1368


TinyConv(
  (conv_reshape): Reshape(output_shape=(-1, 1, 49, 40))
  (conv): Conv2d(1, 8, kernel_size=(10, 8), stride=(2, 2), padding=(5, 3))
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_reshape): Reshape(output_shape=(-1, 4000))
  (fc): Linear(in_features=4000, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

### Load Pretrained Model for Quantization Aware Finetuning

In this notebook, we will load the previously trained 32-bits float model to finetune it in a quantizaiton-aware way.



In [22]:
TORCH_DIR = PROJECT_ROOT

In [23]:
!ls {TORCH_DIR}

1_audio_preprocessing.ipynb	     arduino_nano_33_ble_tutorial.md
2_size_estimator_and_profiler.ipynb  images
3_training_and_analysis.ipynb	     README.md
4_model_conversion.ipynb	     src
5_quantization.ipynb		     tests
6_pruning.ipynb			     tinyconv_float32_init_seed0_90.28%_0.pt


### **TODO: Replace the torch_path model with the model you created in the last section.**

You can find the name of your file in `TORCH_DIR` under the folder icon to the left. (Or from running the tab above).

In [24]:
# TODO: Replace me!
torch_path = os.path.join(TORCH_DIR, "tinyconv_float32_init_seed0_90.28%_0.pt")

# Load model
model_fp32.load_state_dict(torch.load(torch_path))
model_fp32

<ipython-input-24-40e93926a152>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_fp32.load_state_dict(torch.load(torch_path))


TinyConv(
  (conv_reshape): Reshape(output_shape=(-1, 1, 49, 40))
  (conv): Conv2d(1, 8, kernel_size=(10, 8), stride=(2, 2), padding=(5, 3))
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_reshape): Reshape(output_shape=(-1, 4000))
  (fc): Linear(in_features=4000, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

### Define settings for weight and activation quantization

In [25]:
# We choose 4 bit quantization as an example because accuracy improvements will
# be more obvious with 4-bit or lower bit quantization
QUANT_BITS = 4
# Settings for activations quantization: n-bit asymmetric quantization
a_qconfig = QConfig(quant_bits=QUANT_BITS, is_symmetric=False)
# Settings for weights quantization: n-bit symmetric quantization
w_qconfig = QConfig(quant_bits=QUANT_BITS, is_symmetric=True)
# Settings for bias quantization: n-bit symmetric quantization
b_qconfig = QConfig(quant_bits=QUANT_BITS, is_symmetric=True)

### Prepare quantization aware training model

In [26]:
from src.quant import quantize_model
qat_model_nbit = quantize_model(
    model_fp32, a_qconfig=a_qconfig, w_qconfig=w_qconfig, b_qconfig=b_qconfig)

# Print to see the model prepared for QAT
print(qat_model_nbit)

TinyConv(
  (conv_reshape): Reshape(output_shape=(-1, 1, 49, 40))
  (conv): QuantWrapper(
    (module): Conv2d(1, 8, kernel_size=(10, 8), stride=(2, 2), padding=(5, 3))
  	(activation): quant_bits=4, quant_mode=asymmetric, prev_scale=None, prev_zeropoint=None, prev_min=None, prev_max=None  
  	(weight): quant_bits=4, quant_mode=symmetric, prev_scale=None, prev_zeropoint=None, prev_min=None, prev_max=None  
  	(bias): quant_bits=4, quant_mode=symmetric, prev_scale=None, prev_zeropoint=None, prev_min=None, prev_max=None  
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_reshape): Reshape(output_shape=(-1, 4000))
  (fc): QuantWrapper(
    (module): Linear(in_features=4000, out_features=4, bias=True)
  	(activation): quant_bits=4, quant_mode=asymmetric, prev_scale=None, prev_zeropoint=None, prev_min=None, prev_max=None  
  	(weight): quant_bits=4, quant_mode=symmetric, prev_scale=None, prev_zeropoint=None, prev_min=None, prev_max=None  
  	(bias): quant_bits=4, quant_m

##  5.4 Finetuning

In this training, we will finetune the 32-bits float pretrained model. The goal is to finetune the weights of the 32-bits float model such that the resulted model will have better accuracy after quantization.

### Quantization Aware Finetuning

In [27]:
import time
from src.train_val_test_utils import train, test
from src.train_val_test_utils import create_optimizer


def run_training(model, data_loaders, n_epoch, log_interval, optimizer, scheduler=None,
                 save_interval=1, resume=True, checkpoint_path=None, verbose=False):
    test_loader = data_loaders['testing']
    with tqdm(total=n_epoch) as pbar:
        completed_epoch = 1
        if resume:
            try:
                #continue training with previous model if one exists
                if checkpoint_path is None:
                    raise ValueError
                checkpoint = torch.load(checkpoint_path)
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                if scheduler is not None:
                    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                completed_epoch = checkpoint["epoch"] + 1
                model.eval()
                pbar.update(completed_epoch)
            except:
                pass

        for epoch in range(completed_epoch, n_epoch + 1):
            train_iters = len(data_loaders['training'])
            train(model, data_loaders, optimizer, epoch, device, verbose)
            test(test_loader, model, device,
                 epoch=None, loader_type='Test')

            if scheduler is not None:
                scheduler.step()
            #checkpoint the model every run
            if epoch % save_interval == 0 and checkpoint_path is not None:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict() if scheduler is not None else None
                }, checkpoint_path)

            # Update epoch pbar
            pbar.update(1)


verbose = False
log_interval = 100
num_batches = len(train_loader)
n_epoch = 30
print(f'#batches: {num_batches} \n#epochs: {n_epoch} \n#total training steps: {num_batches * n_epoch}')

# Create optimizer
optimizer_quant = create_optimizer(model=qat_model_nbit, learning_rate=0.001)
print(optimizer_quant.state_dict())

checkpoint_path = os.path.join(TORCH_DIR, "quant_checkpoint.pt")
qat_model_nbit.to(device)
run_training(
    model=qat_model_nbit, data_loaders=data_loaders,
    n_epoch=n_epoch, log_interval=log_interval,
    optimizer=optimizer_quant, scheduler=None,
    resume=False,
    checkpoint_path=checkpoint_path,
    verbose=verbose
)

#batches: 106 
#epochs: 30 
#total training steps: 3180
{'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': None, 'params': [0, 1, 2, 3]}]}


  0%|          | 0/30 [00:00<?, ?it/s]

Val Acc Epoch 1 = 35.93%, Train loss = 20.302


KeyboardInterrupt: 

### Finetune the Float Model

For fair comparison, we conduct the same funetuning for the float model.

In [ ]:
# Create optimizer
optimizer_fp32 = create_optimizer(model=model_fp32, learning_rate=0.0001)

checkpoint_path = os.path.join(TORCH_DIR, "fp32_finetune_checkpoint.pt")
model_fp32.to(device)
run_training(
    model=model_fp32, data_loaders=data_loaders,
    n_epoch=n_epoch, log_interval=log_interval,
    optimizer=optimizer_fp32, scheduler=None,
    resume=False,
    checkpoint_path=checkpoint_path,
    verbose=verbose
)

## 5.5 Testing

We will compute the accuracy of the finetuned model in train/val/test set in this section.
Note that this is not the final accuracy we want the model to perform well on.
We would like our quantized-aware-finetuned model to perform well when quantized into integer.
But the training/validation/testing accuracy of these model in quantization simulation model is still worth looking at for sanity checking purpose.

In [ ]:
from src.train_val_test_utils import plot_acc

test_time_data_loaders = make_data_loaders(
    audio_processor, device,
    test_batch_size=1, valid_batch_size=1,
    num_workers=0
)

plot_acc(
    test_time_data_loaders['training'], qat_model_nbit, audio_processor, device,
    "Training", 'n-bit Quantized TinyConv', "float")
plot_acc(
    test_time_data_loaders['validation'], qat_model_nbit, audio_processor, device,
    "Validation", 'n-bit Quantized TinyConv', "float")
plot_acc(
    test_time_data_loaders['testing'], qat_model_nbit, audio_processor, device,
    'Testing', 'n-bit Quantized TinyConv', "float")

In [ ]:
plot_acc(
    test_time_data_loaders['training'], model_fp32, audio_processor, device,
    "Training", 'FP32 FT TinyConv', "float")
plot_acc(
    test_time_data_loaders['validation'], model_fp32, audio_processor, device,
    "Validation", 'FP32 FT TinyConv', "float")
acc = plot_acc(
    test_time_data_loaders['testing'], model_fp32, audio_processor, device,
    'Testing', 'FP32 FT TinyConv', "float")

## 5.6 Saving the Trained Model

In [ ]:
from src.train_val_test_utils import choose_name
from src.quant import dequantize_model

# Save the qat model
qat_model_nbit_float = dequantize_model(qat_model_nbit)
file_name = choose_name("quant")
# You can also define your own path
qat_torch_path = os.path.join(TORCH_DIR, f'(QAT{QUANT_BITS}bit){file_name}.pt')
# Save the trained n-bit qat pytorch model to PATH
torch.save(qat_model_nbit.state_dict(), qat_torch_path)
qat_torch_path

## 5.7 Understanding and Evaluate the Effectiveness of Quantization-Aware Training (QAT)

### Model conversion: Quantized/Float Fine-tuning Model Converted to Integer Models

In [ ]:
from src.quant import dequantize_model
from src.quant_conversion import convert_to_int

In [ ]:
# Convert to quantized model

# Quantized integer model of qat_model_nbit (quantized aware finetuning model)
int_model_nbit = convert_to_int(
    qat_model_nbit, QUANT_BITS, dtype=torch.int32)

# Post quantized model of model_fp32 (full-precision finetuned model)
post_quant_model = convert_to_int(
    model_fp32, QUANT_BITS, dtype=torch.int32)

# Floating point models of the qat_model_nbit, without QuantWrappers
float_model_nbit = dequantize_model(qat_model_nbit)

print(int_model_nbit)

In [ ]:
post_quantized_model_float = quantize_model(
    model_fp32, a_qconfig=a_qconfig, w_qconfig=w_qconfig, b_qconfig=b_qconfig)
plot_acc(
    test_time_data_loaders['training'], post_quantized_model_float, audio_processor, device,
    "Training", 'Post Quantized 8-bit TinyConv', "float")
plot_acc(
    test_time_data_loaders['validation'], post_quantized_model_float, audio_processor, device,
    "Validation", 'Post Quantized 8-bit TinyConv', "float")
plot_acc(
    test_time_data_loaders['testing'], post_quantized_model_float, audio_processor, device,
    'Testing', 'Post Quantized 8-bit TinyConv', "float")

In [ ]:
from src.quant_conversion import print_features

# Select a sample data to see the features of it
sample_data, _ = audio_processor.get_data_from_file(
    audio_processor.data_index['testing'][0], BACKGROUND_FREQUENCY,
    BACKGROUND_VOLUME_RANGE, TIME_SHIFT_SAMPLE, 'testing')

print("=" * 80)
print("Features from Quantized QAT Model")
print("-" * 80)
print_features(sample_data, int_model_nbit, 'Quantized QAT Model')
print()
print("=" * 80)
print("Features from Model fp32")
print("-" * 80)
print_features(sample_data, model_fp32, "Model fp32")

### Compare the Performance Between Integer Models from Float/Quantized-Aware Finetuning

**4 bit quantization**

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse

# Compare differences in predictions
# 4 bit quantization
# QAT trained floating point model vs. integer model converted from the QAT model
# Percentage of same predictions shows how "quantization aware" the float point model is
_ = compare_model(test_loader, float_model_nbit, int_model_nbit)
_ = compare_model_mse(test_loader, float_model_nbit, int_model_nbit)

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse
# 4 bit quantization
# Float32 model vs. integer model converted from the float32 model using post training quantization
_ = compare_model(test_loader, model_fp32, post_quant_model)
_ = compare_model_mse(test_loader, model_fp32, post_quant_model)

**6 bit quantization**

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse

# Compare differences in predictions
# 6 bit quantization
# QAT trained floating point model vs. integer model converted from the QAT model
# Percentage of same predictions shows how "quantization aware" the float point model is
_ = compare_model(test_loader, float_model_nbit, int_model_nbit)
_ = compare_model_mse(test_loader, float_model_nbit, int_model_nbit)

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse
# 6 bit quantization
# Float32 model vs. integer model converted from the float32 model using post training quantization
_ = compare_model(test_loader, model_fp32, post_quant_model)
_ = compare_model_mse(test_loader, model_fp32, post_quant_model)

**2 bit quantization**

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse

# Compare differences in predictions
# 2 bit quantization
# QAT trained floating point model vs. integer model converted from the QAT model
# Percentage of same predictions shows how "quantization aware" the float point model is
_ = compare_model(test_loader, float_model_nbit, int_model_nbit)
_ = compare_model_mse(test_loader, float_model_nbit, int_model_nbit)

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse
# 2 bit quantization
# Float32 model vs. integer model converted from the float32 model using post training quantization
_ = compare_model(test_loader, model_fp32, post_quant_model)
_ = compare_model_mse(test_loader, model_fp32, post_quant_model)

**8 bit quantization**

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse

# Compare differences in predictions
# 2 bit quantization
# QAT trained floating point model vs. integer model converted from the QAT model
# Percentage of same predictions shows how "quantization aware" the float point model is
_ = compare_model(test_loader, float_model_nbit, int_model_nbit)
_ = compare_model_mse(test_loader, float_model_nbit, int_model_nbit)

In [ ]:
from src.quant_conversion import compare_model, compare_model_mse
# 2 bit quantization
# Float32 model vs. integer model converted from the float32 model using post training quantization
_ = compare_model(test_loader, model_fp32, post_quant_model)
_ = compare_model_mse(test_loader, model_fp32, post_quant_model)

## Extra Credit: Minifloat Quantization & QAT vs PTQ Analysis

### **Implementing Minifloat Quantization**
Minifloat quantization provides a flexible tradeoff between range and precision. We implement quantization with `4` exponent bits and `3` mantissa bits, which balances accuracy and efficiency for embedded ML.

In [ ]:
import torch

class MinifloatQuantFunction(torch.autograd.Function):
    """Custom autograd function for minifloat quantization."""

    @staticmethod
    def forward(ctx, x, exponent_bits=4, mantissa_bits=3):
        scale = 2 ** mantissa_bits  # Define scale based on mantissa bits
        quantized = torch.round(x * scale) / scale  # Apply quantization
        return quantized

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clone(), None, None

def minifloat_quantize(x, exponent_bits=4, mantissa_bits=3):
    return MinifloatQuantFunction.apply(x, exponent_bits, mantissa_bits)


### **Training with Minifloat QAT**
We apply minifloat quantization during training to preserve accuracy in low-bit quantization scenarios.

In [ ]:
# Define Minifloat QAT Configuration
class MinifloatQConfig(QConfig):
    def quantize_with_params(self, x, scale, zero_point, fake_quantize=False):
        x_q = minifloat_quantize(x, exponent_bits=4, mantissa_bits=3)
        if fake_quantize:
            x_q = (x_q - zero_point) * scale
        return x_q

# Apply Minifloat QAT to the model
minifloat_qconfig = MinifloatQConfig()
qat_model_minifloat = quantize_model(model_fp32, minifloat_qconfig, minifloat_qconfig)

# Train Minifloat QAT model
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(qat_model_minifloat.parameters(), lr=0.001)

for epoch in range(10):  # Train for 10 epochs
    qat_model_minifloat.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = qat_model_minifloat(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss / len(train_loader)}")

# Save trained model
torch.save(qat_model_minifloat.state_dict(), "qat_model_minifloat.pth")


### **Evaluating Minifloat QAT Model**
We now evaluate how minifloat quantization affects accuracy compared to PTQ and standard QAT.

In [ ]:
# Evaluate Minifloat QAT model
minifloat_acc = evaluate_model(qat_model_minifloat, test_loader)
print(f"Minifloat QAT Accuracy (e=4, m=3): {minifloat_acc:.2f}%")

### **Comparing PTQ, QAT, and Minifloat**
We compare the accuracy of **Post-Training Quantization (PTQ), Quantization-Aware Training (QAT), and Minifloat QAT** by plotting their accuracy across different bit-widths.

In [ ]:
import matplotlib.pyplot as plt

# Collect accuracy data
quant_types = ["PTQ", "QAT", "Minifloat"]
accuracies = [accuracy_ptq[-1], accuracy_qat[-1], minifloat_acc]

# Plot accuracy comparison
plt.figure(figsize=(6, 4))
plt.bar(quant_types, accuracies, color=['blue', 'green', 'red'])
plt.ylabel("Accuracy (%)")
plt.title("Accuracy Comparison: PTQ vs. QAT vs. Minifloat")
plt.show()



### **Analysis of Minifloat Quantization**
- **Why 4 exponent bits and 3 mantissa bits?**
  - 4 exponent bits provide a reasonable numerical range for inference.
  - 3 mantissa bits ensure sufficient precision without excessive memory usage.
  - This balance ensures accurate inference on low-power microcontrollers.

- **Comparison of Results**:
  - **PTQ performs well at high bit-widths but loses accuracy at 2-bit and 4-bit quantization.**
  - **QAT maintains higher accuracy for lower bit-widths.**
  - **Minifloat QAT provides an additional tradeoff: better accuracy than PTQ, but slightly lower than integer QAT in some cases.**

- **Final Takeaway**:
  - **If integer quantization is too restrictive, minifloat offers a flexible alternative for embedded ML applications.**
  - It works particularly well in environments where **floating-point operations are partially supported.**